In [50]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [51]:
def sample_size_calc_mean(control_mean,std_dev,relative_change,control_size):
    
    test_mean = control_mean*(1+(relative_change/100))
    effective_size = (test_mean - control_mean)/std_dev
    
    k=0
    
    l = [control_size,100-control_size]
    
    for i in l:

        required_n = sms.NormalIndPower().solve_power(effective_size, 
        power=0.8, 
        alpha=0.05, 
        ratio=i/(100-i),
        alternative='larger')                                                 

        required_n = ceil(required_n)
        output_mean['Change']=relative_change
        

        if k%2==0:
            output_mean.loc[len(output_mean.index)] = ['test',100-i,required_n]
        else:
            output_mean.loc[len(output_mean.index)] =['control',100-i,required_n] 

        k+=1

In [52]:
def sample_size_calc_mean(control_mean,std_dev,relative_change,control_size):
    
    test_mean = control_mean*(1+(relative_change/100))
    effective_size = (test_mean - control_mean)/std_dev
    alpha=0.05
    power=0.8
    
    required_n_test = sms.NormalIndPower().solve_power(effective_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=control_size/(100-control_size),
    alternative='larger')                                                 
    required_n_test = ceil(required_n_test)
    
    required_n_control = sms.NormalIndPower().solve_power(effective_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=(100-control_size)/control_size,
    alternative='larger')                                                 
    required_n_control = ceil(required_n_control)
    
    output_mean['Change']=[relative_change]
    output_mean['Split']=[str(100-control_size)+'-'+str(control_size)]
    output_mean['Test_volume']=[required_n_test]
    output_mean['Control_volume']=[required_n_control]
    output_mean['Test_mean']=[test_mean]
    output_mean['Control_mean']=[control_mean]
    output_mean['Expected STDDEV']=[std_dev]
    output_mean['Test_claims']=[ceil(required_n_test*test_mean)]
    output_mean['control_claims']=[ceil(required_n_control*control_mean)]

In [53]:
output_mean.columns

Index(['Change', 'Split', 'Test_volume', 'Control_volume', 'Test_mean',
       'Control_mean', 'Expected STDDEV', 'Test_claims', 'control_claims'],
      dtype='object')

In [55]:
final_output=pd.DataFrame()
#output_mean = pd.DataFrame()
for i in [2,5,10,25,50]:
    output_mean = pd.DataFrame()
    control_mean = 2.80
    std_dev = .66
    relative_change = 1
    control_size = i     #control percentage 
    sample_size_calc_mean(control_mean,std_dev,relative_change,control_size)
    final_output= pd.concat([final_output,output_mean])

In [56]:
final_output

,Change,Split,Test_volume,Control_volume,Test_mean,Control_mean,Expected STDDEV,Test_claims,control_claims
0,1,98-2,171756,3506,2.828,2.8,0.66,485726,9817
0,1,95-5,68703,3616,2.828,2.8,0.66,194293,10125
0,1,90-10,34352,3817,2.828,2.8,0.66,97148,10688
0,1,75-25,13741,4581,2.828,2.8,0.66,38860,12827
0,1,50-50,6871,6871,2.828,2.8,0.66,19432,19239


In [ ]:
 z_critical = stats.norm.ppf(1-alpha/2)
    ncp = stats.norm.ppf(power) - stats.norm.ppf(1 - power) * 0.5
    test_rate = 1 - stats.norm.cdf(z_critical - ncp)
    

In [93]:
def sample_size_calc_prop(control_resp_rate,relative_change,control_size):
    
    test_resp_rate = control_resp_rate*(1+(relative_change/100))
    effect_size = sms.proportion_effectsize(control_resp_rate,test_resp_rate)  

    required_n_test = sms.NormalIndPower().solve_power(-effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=control_size/(100-control_size),
    alternative='larger')                                                 

    required_n_test = ceil(required_n_test)

    required_n_control = sms.NormalIndPower().solve_power(-effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=(100-control_size)/control_size,
    alternative='larger')     
    
    test_rate =control_resp_rate + (effect_size * (1 - control_resp_rate))

    output_prop['Change']=[relative_change]
    output_prop['Split']=[str(100-control_size)+'-'+str(control_size)]
    output_prop['Test_volume']=[required_n_test]
    output_prop['control_volume']=[required_n_control]
    output_prop['Test_rate']=[test_rate]
    output_prop['Control_rate']=control_resp_rate
    output_prop['Test_claims']=[ceil(required_n_test*test_rate)]
    output_prop['Control_claims']=[ceil(required_n_control*control_resp_rate)]
    output_prop['Test_LTV']=ceil(output_prop['Test_claims']*LTV)
    output_prop['Control_LTV']=ceil(output_prop['Control_claims']*LTV)

In [98]:
final_output_prop=pd.DataFrame()
for i in [2,5,10,25,50]:
    output_prop = pd.DataFrame()
    control_resp_rate = 0.552
    relative_change = 1
    control_size = i
    alpha=0.05
    power=0.8
    LTV=10
    sample_size_calc_prop(control_resp_rate,relative_change,control_size)
    final_output_prop=pd.concat([final_output_prop,output_prop])

In [99]:
final_output_prop

,Change,Split,Test_volume,control_volume,Test_rate,Control_rate,Test_claims,Control_claims,Test_LTV,Control_LTV
0,1,98-2,2505848,51139.749984,0.547024,0.552,1370760,28230,13707600,282300
0,1,95-5,1002340,52754.689457,0.547024,0.552,548305,29121,5483050,291210
0,1,90-10,501170,55685.505538,0.547024,0.552,274153,30739,2741530,307390
0,1,75-25,200468,66822.606646,0.547024,0.552,109661,36887,1096610,368870
0,1,50-50,100234,100233.909969,0.547024,0.552,54831,55330,548310,553300


In [100]:
final_output

,Change,Split,Test_volume,Control_volume,Test_mean,Control_mean,Expected STDDEV,Test_claims,control_claims
0,1,98-2,171756,3506,2.828,2.8,0.66,485726,9817
0,1,95-5,68703,3616,2.828,2.8,0.66,194293,10125
0,1,90-10,34352,3817,2.828,2.8,0.66,97148,10688
0,1,75-25,13741,4581,2.828,2.8,0.66,38860,12827
0,1,50-50,6871,6871,2.828,2.8,0.66,19432,19239


In [123]:
from scipy.stats import norm
import math
lift = 0.04
test_control_split=0.5
control = 1 - test_control_split
z_alpha = norm.ppf(1 - 0.05/2)
z_beta = norm.ppf(1 - 0.2)
p1 = 0.5 + lift/2
p2 = 0.5 - lift/2
p_pool = control*p1 + test_control_split*p2
print(p1,p2,p_pool)
n = (z_alpha*math.sqrt(2*p_pool*(1 - p_pool)) + z_beta*math.sqrt(p1*(1 - p1) + p2*(1 - p2)))**2/(lift**2)
power = 1 - norm.cdf((z_alpha*math.sqrt(n*(p1 - p2)**2) + (p1 - p2)*n)/(math.sqrt(p_pool*(1 - p_pool))*math.sqrt(n)))

0.52 0.48 0.5


In [121]:
print('Power: {:.3f}'.format(power))

Power: 0.000


In [164]:
import numpy as np
from statsmodels.stats.power import TTestIndPower
control_mean = 2.8
control_std = 0.66

relative_change = .04

test_control_split = 0.4

experimental_mean = control_mean * (1 + relative_change)
experimental_std = control_std

pooled_std = np.sqrt(((test_control_split * control_std ** 2) + ((1 - test_control_split) * experimental_std ** 2)) / (test_control_split + (1 - test_control_split)))

effect_size = (experimental_mean - control_mean) / pooled_std

nobs = 515

# Calculate the power using TTestIndPower
power_analysis = TTestIndPower()
power = power_analysis.power(effect_size=effect_size, nobs1=nobs * test_control_split, alpha=0.05, ratio=1 / test_control_split, alternative='two-sided')

print('Power: {:.3f}'.format(power*100))



Power: 53.817
